In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 2.1 Constant-velocity (CV) extrapolation

In [2]:
import pandas as pd
import numpy as np
import glob
from tqdm import tqdm
import pickle, os

save_dir = "/content/drive/MyDrive/NFL Big Data Bowl 2026/preprocessed"
df_in_path  = os.path.join(save_dir, "df_in_norm.pkl")
df_out_path = os.path.join(save_dir, "df_out_norm.pkl")

print("📂 Loading preprocessed normalized data...")
df_in_norm  = pd.read_pickle(df_in_path)
df_out_norm = pd.read_pickle(df_out_path)
print(f"✅ Loaded: {len(df_in_norm):,} input rows | {len(df_out_norm):,} output rows")

FIELD_X_MIN, FIELD_X_MAX = 0.0, 120.0
FIELD_Y_MIN_C, FIELD_Y_MAX_C = -26.65, 26.65

def constant_velocity_forecast(df_in_norm: pd.DataFrame, df_out_norm: pd.DataFrame, k_last: int = 3):
    df_in = df_in_norm.sort_values(["game_id","play_id","nfl_id","frame_id"]).copy()
    # yards per frame (10 Hz → DO NOT multiply by 10)
    df_in["vx"] = df_in.groupby(["game_id","play_id","nfl_id"])["x"].diff().fillna(0.0)
    df_in["vy"] = df_in.groupby(["game_id","play_id","nfl_id"])["y"].diff().fillna(0.0)

    # last position at throw and mean of last k velocities
    lastK = (
        df_in.groupby(["game_id","play_id","nfl_id"], as_index=False)
             .apply(lambda g: pd.Series({
                 "x0": g["x"].iloc[-1],
                 "y0": g["y"].iloc[-1],
                 "vx": g["vx"].tail(k_last).mean(),
                 "vy": g["vy"].tail(k_last).mean(),
             }))
             .reset_index(drop=True)
    )

    preds = []
    for (gid,pid), g in df_out_norm.groupby(["game_id","play_id"]):
        frames = np.array(sorted(g["frame_id"].unique()))
        t = np.arange(1, len(frames)+1, dtype=float)  # 1,2,... frames since throw
        players = lastK.query("game_id==@gid and play_id==@pid")
        if players.empty:
            continue
        for _, r in players.iterrows():
            x = r.x0 + r.vx * t
            y = r.y0 + r.vy * t
            x = np.clip(x, FIELD_X_MIN, FIELD_X_MAX)
            y = np.clip(y, FIELD_Y_MIN_C, FIELD_Y_MAX_C)
            preds.append(pd.DataFrame({
                "game_id": gid, "play_id": pid, "nfl_id": r.nfl_id,
                "frame_id": frames, "x": x, "y": y
            }))
    return pd.concat(preds, ignore_index=True) if preds else pd.DataFrame(columns=["game_id","play_id","nfl_id","frame_id","x","y"])

# ================= Apply to all weeks =================
preds = constant_velocity_forecast(df_in_norm, df_out_norm, k_last=3)

merged = df_out_norm.merge(preds, on=["game_id","play_id","nfl_id","frame_id"],
                           suffixes=("_true","_pred"))
merged["err"] = np.sqrt((merged.x_true - merged.x_pred)**2 + (merged.y_true - merged.y_pred)**2)

overall_rmse = merged["err"].mean()
print(f"🏁 Constant-velocity baseline RMSE: {overall_rmse:.3f} yd")

# Optional: per-week diagnostics
if "week" in df_in_norm.columns:
    rmse_weeks = merged.groupby("week")["err"].mean().reset_index()
    print("\n📊 RMSE per week:")
    print(rmse_weeks)

📂 Loading preprocessed normalized data...
✅ Loaded: 1,303,440 input rows | 562,936 output rows


/tmp/ipython-input-813696132.py:28: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({


🏁 Constant-velocity baseline RMSE: 1.697 yd


# 2.2 Kalman (CV + coordinated-turn + goal attraction)

In [3]:
import numpy as np
import pandas as pd
import pickle, os
from tqdm import tqdm

FIELD_X_MIN, FIELD_X_MAX = 0.0, 120.0
FIELD_Y_MIN_C, FIELD_Y_MAX_C = -26.65, 26.65

# ------------------------------------------------------------
# Throw snapshot + kinematics (yards/frame) + yaw-rate (rad/frame)
# ------------------------------------------------------------
def _unwrap(a):
    return np.unwrap(a)

def throw_snapshot_and_kinematics(df_in_norm: pd.DataFrame,
                                  k_last_vel: int = 3,
                                  yaw_window: int = 5) -> pd.DataFrame:
    """
    One row per (game_id, play_id, nfl_id) at the throw frame with:
      x0,y0, vx,vy (mean of last k), omega (yaw-rate rad/frame), and role/side.
    """
    df = df_in_norm.sort_values(["game_id","play_id","nfl_id","frame_id"]).copy()
    keys = ["game_id","play_id","nfl_id"]

    # per-frame velocities (yards/frame)
    df["vx"] = df.groupby(keys)["x"].diff().fillna(0.0)
    df["vy"] = df.groupby(keys)["y"].diff().fillna(0.0)

    def per_player(g):
        x0, y0 = g["x"].iloc[-1], g["y"].iloc[-1]
        vx = g["vx"].tail(k_last_vel).mean()
        vy = g["vy"].tail(k_last_vel).mean()

        # yaw-rate (coordinated-turn): unwrap heading over last yaw_window frames
        vx_series = g["vx"].tail(yaw_window).values
        vy_series = g["vy"].tail(yaw_window).values
        headings = np.arctan2(vy_series, vx_series)
        headings = _unwrap(headings)
        if len(headings) >= 2:
            omega = np.mean(np.diff(headings[-3:])) if len(headings) >= 4 else np.median(np.diff(headings))

            if not np.isfinite(omega):
                omega = 0.0
        else:
            omega = 0.0

        # fallback if vx,vy are both ~0 but s/dir exist
        if abs(vx) + abs(vy) < 1e-6 and {"s","dir"}.issubset(g.columns):
            # s is yards/sec; convert to yards/frame by dividing by 10
            s_last = g["s"].iloc[-1] / 10.0
            dir_rad = np.deg2rad(g["dir"].iloc[-1])
            vx, vy = s_last*np.cos(dir_rad), s_last*np.sin(dir_rad)

        side = g["player_side"].iloc[-1] if "player_side" in g.columns else None
        role = g["player_role"].iloc[-1] if "player_role" in g.columns else None

        return pd.Series({"x0":x0,"y0":y0,"vx":vx,"vy":vy,"omega":omega,
                          "player_side":side,"player_role":role})

    snap = df.groupby(keys, as_index=False).apply(per_player).reset_index(drop=True)
    return snap

# ------------------------------------------------------------
# Goal attraction selection (Targeted WR + nearest 1–2 defenders)
# ------------------------------------------------------------
def select_goal_units(df_in_norm: pd.DataFrame, snap: pd.DataFrame,
                      n_defenders: int = 2) -> pd.DataFrame:
    """
    Returns a table with an 'goal_gain' column per player:
      - Targeted WR: 0.10 yd/frame^2 (== 1.0 yd/s^2)
      - Nearest n defenders to the WR at throw: 0.06 yd/frame^2
      - Others: 0.0
    """
    keys_gp = ["game_id","play_id"]
    # ball landing per play
    ball_land = (
        df_in_norm.groupby(keys_gp, as_index=False)
        .apply(lambda g: pd.Series({
            "ball_land_x": g["ball_land_x"].iloc[-1] if "ball_land_x" in g else np.nan,
            "ball_land_y": g["ball_land_y"].iloc[-1] if "ball_land_y" in g else np.nan
        }))
        .reset_index(drop=True)
    )

    # get throw positions (x0,y0) from snap
    trk = snap.merge(ball_land, on=keys_gp, how="left")

    # find targeted WR per play
    targets = snap.query("player_role == 'Targeted Receiver'")[["game_id","play_id","nfl_id"]]
    if targets.empty:
        trk["goal_gain"] = 0.0
        return trk

    # merge target to get their position
    trk = trk.merge(targets.rename(columns={"nfl_id":"target_id"}),
                    on=keys_gp, how="left")

    gains = []
    for (gid,pid), g in trk.groupby(keys_gp):
        g = g.copy()
        # target row
        tgt_row = g[g["nfl_id"]==g["target_id"].iloc[0]] if g["target_id"].notna().any() else pd.DataFrame()
        # default zero
        g["goal_gain"] = 0.0

        if not tgt_row.empty:
            # Targeted WR gets 0.10
            g.loc[g["nfl_id"]==tgt_row["nfl_id"].iloc[0], "goal_gain"] = 0.10

            # nearest defenders get 0.06
            # throw positions from snap: x0,y0
            tr_x, tr_y = tgt_row["x0"].iloc[0], tgt_row["y0"].iloc[0]
            defenders = g[g["player_side"]=="Defense"].copy()
            if not defenders.empty:
                defenders["dist"] = np.hypot(defenders["x0"]-tr_x, defenders["y0"]-tr_y)
                near_ids = defenders.nsmallest(n_defenders, "dist")["nfl_id"].tolist()
                g.loc[g["nfl_id"].isin(near_ids), "goal_gain"] = 0.06

        gains.append(g)

    return pd.concat(gains, ignore_index=True)

# ------------------------------------------------------------
# Coordinated-turn + goal-attraction predictor
# ------------------------------------------------------------
def predict_kalman_ct_goal(df_in_norm: pd.DataFrame,
                           df_out_norm: pd.DataFrame,
                           k_last_vel: int = 3,
                           yaw_window: int = 5,
                           n_defenders: int = 2,
                           omega_clip: float = 0.15):
    """
    Returns predictions with columns: game_id, play_id, nfl_id, frame_id, x, y
    """
    # snapshot at throw
    snap = throw_snapshot_and_kinematics(df_in_norm, k_last_vel=k_last_vel, yaw_window=yaw_window)
    # limit extreme yaw
    snap["omega"] = snap["omega"].clip(-omega_clip, omega_clip)

    # hook in goal gains
    snap_g = select_goal_units(df_in_norm, snap, n_defenders=n_defenders)

    preds = []
    for (gid,pid), g_out in df_out_norm.groupby(["game_id","play_id"]):
        frames = np.array(sorted(g_out["frame_id"].unique()))
        T = len(frames)
        if T == 0:
            continue

        players = snap_g.query("game_id==@gid and play_id==@pid").copy()
        if players.empty:
            continue

        bx = players["ball_land_x"].iloc[0] if "ball_land_x" in players else np.nan
        by = players["ball_land_y"].iloc[0] if "ball_land_y" in players else np.nan
        has_ball = np.isfinite(bx) and np.isfinite(by)


        for _, r in players.iterrows():
            if abs(r.vx) < 0.05 and abs(r.vy) < 0.05:
                # barely moving → predict constant position
                preds.append(pd.DataFrame({
                    "game_id": gid, "play_id": pid, "nfl_id": r.nfl_id,
                    "frame_id": frames, "x": r.x0, "y": r.y0
                }))
                continue

            x, y = r.x0, r.y0
            vx, vy = r.vx, r.vy
            omega = r.omega
            gain = r.goal_gain  # yd/frame^2

            traj_x = np.empty(T, dtype=float)
            traj_y = np.empty(T, dtype=float)

            for t in range(T):
                # 1) coordinated turn (rotate velocity by omega each frame)
                if abs(omega) > 1e-6:
                    c, s = np.cos(omega), np.sin(omega)
                    vx, vy = c*vx - s*vy, s*vx + c*vy  # rotate

                # 2) goal attraction (small acceleration toward landing)
                if has_ball and gain > 0.0:
                    dx, dy = bx - x, by - y
                    L = np.hypot(dx, dy) + 1e-6
                    # fade acceleration when close (<15 yds)
                    gain_eff = gain * np.clip(L / 40.0, 0.0, 1.0)
                    ax, ay = gain_eff * (dx/L), gain_eff * (dy/L)
                    vx += ax
                    vy += ay

                # 3) integrate position
                x += vx
                y += vy
                # 4) clip to field
                x = float(np.clip(x, FIELD_X_MIN, FIELD_X_MAX))
                y = float(np.clip(y, FIELD_Y_MIN_C, FIELD_Y_MAX_C))

                traj_x[t] = x
                traj_y[t] = y

            preds.append(pd.DataFrame({
                "game_id": gid,
                "play_id": pid,
                "nfl_id": r.nfl_id,
                "frame_id": frames,
                "x": traj_x,
                "y": traj_y
            }))

    return pd.concat(preds, ignore_index=True) if preds else pd.DataFrame(columns=["game_id","play_id","nfl_id","frame_id","x","y"])

In [4]:
# ------------------------------------------------------------
# 1️⃣ Load preprocessed normalized data
# ------------------------------------------------------------
save_dir = "/content/drive/MyDrive/NFL Big Data Bowl 2026/preprocessed"
df_in_norm  = pd.read_pickle(os.path.join(save_dir, "df_in_norm.pkl"))
df_out_norm = pd.read_pickle(os.path.join(save_dir, "df_out_norm.pkl"))

print(f"✅ Loaded preprocessed data: {len(df_in_norm):,} input rows | {len(df_out_norm):,} output rows")

preds_kf = predict_kalman_ct_goal(df_in_norm, df_out_norm,
                                  k_last_vel=3, yaw_window=5,
                                  n_defenders=2, omega_clip=0.05)

merged = df_out_norm.merge(preds_kf, on=["game_id","play_id","nfl_id","frame_id"],
                           suffixes=("_true","_pred"))
merged["err"] = np.sqrt((merged.x_true - merged.x_pred)**2 + (merged.y_true - merged.y_pred)**2)

rmse = merged["err"].mean()
print(f"🏁 Kalman CT + Goal baseline RMSE: {rmse:.3f} yd")

✅ Loaded preprocessed data: 1,303,440 input rows | 562,936 output rows


/tmp/ipython-input-2158584043.py:60: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  snap = df.groupby(keys, as_index=False).apply(per_player).reset_index(drop=True)
/tmp/ipython-input-2158584043.py:78: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({


🏁 Kalman CT + Goal baseline RMSE: 1.405 yd


KeyError: 'player_role'